# Introduction

In this file, spot light search MVPA is applied on Z-stat files from first level analysis I have done on naive space, ART applied images, with each stimuli giving one Z-stat image.
PLUS: EEG measures are added to the classifers.

MPVA is applied for all voxels of the brain. For each participant, data is split into the number of runs, with data from one run is used for test, and the rest is used for training. 
    

In [1]:
import numpy as np
import pandas as pd
import os.path as op
import matplotlib.pyplot as plt
import nibabel as nib
import itertools
import multiprocessing
from multiprocessing import Pool
from importlib import reload #import function "reload"
import sys
import csv
import warnings
warnings.filterwarnings("ignore")
import os
import glob
from glob import glob
from scipy.io import loadmat
import math
import timeit
from timeit import default_timer as timer

plt.style.use('classic')
%matplotlib inline

In [2]:
from MVPA_func import SpherePoints, SphereValue, LoadAllNii, MVPA_FeatureExtraction, MVPA_Classification, LoopiFeatureDict, multi_run_wrapperLoopiFeatureDict

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
#from sklearn.svm import LinearSVC

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

### Load subject list file

In [4]:
subj_list_file='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/subj_simultaneous.txt'
subj_list=list()
f = open(subj_list_file, 'r')
line=f.readline()
while line:
    subj_list.append(line[0:-1])
    oldline=line
    line = f.readline()
subj_list.pop()
subj_list.append(oldline)
f.close()
Nsubj=len(subj_list)

### Load Partipants' ERP measures


In [5]:
ALLERP=['P3','LPP']
ERPMeasure=['Peak','Mean','Latency']

desire_channel_P1name = []
desire_channel_P3name = ['Cz']#, 'Pz', 'PCz','CL', 'CR', 'PL', 'PR']
desire_channel_LPPname = ['Pz']#, 'PL','PR','T5','T6']


timeseries_path='C:\\Users\\user\\Documents\\Reyhaneh\\EEG+fMRI\\EO\\EEG_preproc\\ts\\'
# List the folders in the specified directory
ALLERP=['P3','LPP']
CSV_out=timeseries_path+"ERP_measures.csv"

ERP_df = pd.read_csv(timeseries_path+"ERP_measures.csv") 

#List of EEG features to be used for classification (can be more/less but should have similar format: ERP_Site_Measure)
#EEG_features=['P3_Cz_Peak']#,'LPP_Pz_Latency']
EEG_features=['LPP_Pz_Peak']#,'LPP_Pz_Latency']


### Load SPM.mat to extract fMRI experiment info
This cell loads spm.mat file for each participant and from the data, extracts the number of runs, the path for each stat file, etc

In [6]:
# Note 3: I don't know why the last item is repeated! but it doesn't make a misatke

conditions=['Neu','Neg','Pos','Targ']
ZstatInfo_file='ZstatInfo.csv'
# if not os.path.exists(ZstatInfo_file):
    
ERP_heading=[]
for myERP in ALLERP:
    desire_channel_ERP_name=getattr(sys.modules[__name__], f"desire_channel_{myERP}name")
    for mySite in desire_channel_ERP_name:
        for myERPMeasure in ERPMeasure:
            #print(f'{myERP}_{mySite}_{myERPMeasure}')
            ERP_heading.append(f'{myERP}_{mySite}_{myERPMeasure}')

ZstatInfo=pd.DataFrame(columns=['Subj','Run','Folder']+conditions+ [condition + '_'+ERP_h for condition in conditions if condition != 'Pos' for ERP_h in ERP_heading])
#subj_list=['ET_12102013']
for subj in subj_list:
    #print('Participant: '+subj)
    # Zstat files and their condisitons are in SPM filesWe repeat loading in the dataframe to avoid dividing the onsets by 2 multiple times ...
    SubjSPMFolder='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/SPM/'+subj+'/3cond_dist2/'
    os.path.isdir(SubjSPMFolder)
    myZstat=[]

    iRuns = 0
    for myRun in os.listdir(SubjSPMFolder):
        if (subj=='ET_12102013') & (myRun=='run01'): 
            continue  
        ZstatInfotmp=pd.DataFrame(columns=ZstatInfo.columns,index=[0])
        ZstatInfotmp['Subj'][0]=subj

        RunSubjSPM=SubjSPMFolder+myRun+'/SPM.mat'
        ZstatInfotmp['Folder'][0]=SubjSPMFolder+myRun+'/'

        #print('   loading SPM.mat for '+myRun+'...')
        mat = loadmat(RunSubjSPM)
        mdata=mat['SPM'][0,0]
        ndata=mdata['Sess'][0,0]
        odata=ndata['U']
        pdata=odata['name'][0]
        ZstatNames=[pdata[i][0,0][0] for i in range(len(pdata))]
        myZstat.append(ZstatNames)

        ZstatInfotmp['Run'][0]=iRuns+1
        old_ind=len(myZstat[iRuns])+1
        for iCond in reversed(conditions):    
            #print(iCond)
            Start_ind=myZstat[iRuns].index(iCond+'_1')+1
            Trial_iCond_len=old_ind-Start_ind+1
            #print(Start_ind)
            ZstatInfotmp[iCond][0]=['spmT_'+str(val).zfill(4)+'.nii' for val in list(range(Start_ind, old_ind))]#list(range(Start_ind, old_ind))

            for myERP in ALLERP:
                desire_channel_ERP_name=getattr(sys.modules[__name__], f"desire_channel_{myERP}name")
                for mySite in desire_channel_ERP_name:
                    for myERPMeasure in ERPMeasure:
                       # print(f'{myERP}_{mySite}_{myERPMeasure}')
                        if iCond != 'Pos':
                            ERP_Cond=[]
                            for iTrial in range(1,Trial_iCond_len):
                                RowVal=ERP_df.loc[(ERP_df['Subject_ID'] == subj) & (ERP_df['Run'] == myRun) & (ERP_df['ERP'] == myERP) & (ERP_df['Site'] == mySite) & (ERP_df['Condition']==iCond+'_'+str(iTrial)),myERPMeasure]
                                try:
                                    ERP_Cond.append(RowVal.values[0])
                                except IndexError:
                                    pass
        #                     print(iCond)
        #                     print(ERP_Cond)
                            ZstatInfotmp[f'{iCond}_{myERP}_{mySite}_{myERPMeasure}'][0]=ERP_Cond
            old_ind=Start_ind

        ZstatInfo=ZstatInfo.append(ZstatInfotmp,ignore_index=True)
        iRuns = iRuns+1



# Create a new Distractor Columns
# Function to concatenate lists
def combine_lists(row, Neu, Neg):
    return row[Neu] + row[Neg]

# Apply the function to each row
ZstatInfo['Distractor'] = ZstatInfo.apply(combine_lists, args=('Neu', 'Neg'), axis=1)
for ERP_h in ERP_heading:
    ZstatInfo['Distractor'+'_'+ERP_h] = ZstatInfo.apply(combine_lists, args=('Neu'+'_'+ERP_h, 'Neg'+'_'+ERP_h), axis=1)

ZstatInfo.to_csv('ZstatInfo.csv', index=False) 
# ZstatInfo = pd.read_csv(ZstatInfo_file) 

,Subj,Run,Folder,Neu,Neg,Pos,Targ,Neu_P3_Cz_Peak,Neu_P3_Cz_Mean,Neu_P3_Cz_Latency,...,Targ_LPP_Pz_Peak,Targ_LPP_Pz_Mean,Targ_LPP_Pz_Latency,Distractor,Distractor_P3_Cz_Peak,Distractor_P3_Cz_Mean,Distractor_P3_Cz_Latency,Distractor_LPP_Pz_Peak,Distractor_LPP_Pz_Mean,Distractor_LPP_Pz_Latency
0,AB_04152014,1,C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/...,"[spmT_0001.nii, spmT_0002.nii, spmT_0003.nii, ...","[spmT_0007.nii, spmT_0008.nii, spmT_0009.nii, ...",[spmT_0014.nii],"[spmT_0015.nii, spmT_0016.nii, spmT_0017.nii, ...","[9.8755, 24.095, 29.351, 22.103, 29.02, 16.702]","[-4.1597708, 10.165772, 22.0934, 7.4361724, 8....","[500, 500, 268, 428, 520, 252]",...,"[-0.92801, 10.789, 9.9826, 16.31, -1.3609, 2.9...","[-10.62238413, 0.4050916, -3.291955333, -1.536...","[724, 860, 824, 808, 672, 896, 840, 948, 832, ...","[spmT_0001.nii, spmT_0002.nii, spmT_0003.nii, ...","[9.8755, 24.095, 29.351, 22.103, 29.02, 16.702...","[-4.1597708, 10.165772, 22.0934, 7.4361724, 8....","[500, 500, 268, 428, 520, 252, 432, 424, 440, ...","[14.55, -1.3397, 1.2408, 24.771, 17.322, 16.37...","[0.669740133, -9.486729333, -8.3858208, 15.183...","[696, 884, 768, 888, 676, 788, 796, 652, 696, ..."
1,AB_04152014,2,C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/...,"[spmT_0001.nii, spmT_0002.nii, spmT_0003.nii, ...","[spmT_0009.nii, spmT_0010.nii, spmT_0011.nii, ...",[spmT_0017.nii],"[spmT_0018.nii, spmT_0019.nii, spmT_0020.nii, ...","[18.377, 28.673, 14.027, 10.588, 22.404, 19.19...","[1.508489547, 12.13782947, 2.444746344, 0.7709...","[312, 288, 276, 544, 288, 276, 428, 496]",...,"[25.094, 15.209, 30.59, 31.175, 4.7639, 16.763...","[6.758286, 4.677261813, 21.168872, 12.15884147...","[820, 724, 856, 772, 908, 904, 808, 848, 680, ...","[spmT_0001.nii, spmT_0002.nii, spmT_0003.nii, ...","[18.377, 28.673, 14.027, 10.588, 22.404, 19.19...","[1.508489547, 12.13782947, 2.444746344, 0.7709...","[312, 288, 276, 544, 288, 276, 428, 496, 280, ...","[11.949, 7.546, 2.4261, 16.979, 24.493, 3.8052...","[0.027617387, -1.272065467, -9.181824533, 6.94...","[932, 664, 888, 684, 884, 712, 676, 860, 668, ..."
2,AB_04152014,3,C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/...,"[spmT_0001.nii, spmT_0002.nii, spmT_0003.nii, ...","[spmT_0009.nii, spmT_0010.nii, spmT_0011.nii, ...",[spmT_0017.nii],"[spmT_0018.nii, spmT_0019.nii, spmT_0020.nii, ...","[17.642, 16.067, 27.048, 17.067, 9.9208, -5.93...","[-1.979726613, 4.707850133, 16.80802533, 6.434...","[452, 376, 456, 424, 464, 252, 360, 292]",...,"[14.371, 1.5218, 7.7681, 15.377, 19.528, 15.26...","[-1.131164067, -8.2070472, -12.47131, -3.23641...","[908, 804, 728, 896, 920, 728, 776, 900, 656, ...","[spmT_0001.nii, spmT_0002.nii, spmT_0003.nii, ...","[17.642, 16.067, 27.048, 17.067, 9.9208, -5.93...","[-1.979726613, 4.707850133, 16.80802533, 6.434...","[452, 376, 456, 424, 464, 252, 360, 292, 296, ...","[11.859, 20.073, 17.88, 20.149, 22.733, -2.181...","[-1.254729181, 4.558871867, 1.19282252, 6.9493...","[712, 668, 664, 944, 728, 848, 804, 672, 756, ..."
3,AB_04152014,4,C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/...,"[spmT_0001.nii, spmT_0002.nii, spmT_0003.nii, ...","[spmT_0009.nii, spmT_0010.nii, spmT_0011.nii, ...",[spmT_0017.nii],"[spmT_0018.nii, spmT_0019.nii, spmT_0020.nii, ...","[14.463, 8.2032, 26.305, 7.5514, 31.49, 18.695...","[-12.32902525, -3.948568947, 12.053826, -1.330...","[544, 288, 444, 276, 436, 440, 304, 484]",...,"[11.946, 11.919, 9.1433, 1.8999, 4.2575, 19.13...","[-8.830001467, -1.775967067, -3.905192, -5.262...","[748, 844, 940, 916, 884, 784, 716, 928, 700, ...","[spmT_0001.nii, spmT_0002.nii, spmT_0003.nii, ...","[14.463, 8.2032, 26.305, 7.5514, 31.49, 18.695...","[-12.32902525, -3.948568947, 12.053826, -1.330...","[544, 288, 444, 276, 436, 440, 304, 484, 388, ...","[17.548, 21.824, 19.464, 12.826, 23.762, -0.49...","[1.970181693, 1.863903933, 6.49108772, 3.03228...","[672, 784, 868, 720, 720, 796, 680, 720, 764, ..."
4,AB_04152014,5,C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/...,"[spmT_0001.nii, s

### find Voxel dimension by openning one of the image files


In [19]:
MyZstat=ZstatInfo.loc[(ZstatInfo['Subj']==subj) & (ZstatInfo['Run']==1)]
MyZstat.reset_index(inplace = True, drop = True)
MyFile=MyZstat['Folder'][0]+MyZstat['Neu'][0][0]
#print(MyFile)
data = nib.load(MyFile).get_data()
voxel_dims = data.shape
print(voxel_dims)

(64, 64, 28)


In [20]:
# Calculate the spotlights voxels for all points and keep them in AllPoints 
start = timer()
radius = 2 #spheres  radius 

AllPoints_file='Radius'+str(radius)+'_points.npy'
if os.path.exists(AllPoints_file):
    AllPoints=np.load(AllPoints_file)
else:
    tmpPoints = SpherePoints(np.array([26,26,16]) , radius, voxel_dims) #generate a random sphere to find its dimension
    print('size of shperes:',tmpPoints.shape)
    AllZeroSphere = np.zeros((1,tmpPoints.shape[2]))
    AllZeroSpherePoints=np.zeros(tmpPoints.shape)
    cnt=0
    for myCenter in itertools.product(range(0,voxel_dims[0]), range(0,voxel_dims[1]), range(0,voxel_dims[2])):
        #print(myCenter)
        myPoints = SpherePoints(np.array(myCenter) , radius, voxel_dims)
        if myPoints.shape==(3,1,0): myPoints=AllZeroSpherePoints
        if cnt==0:
            AllPoints = np.array(myPoints,ndmin=4)
            cnt=1
        else:
            AllPoints = np.concatenate((AllPoints,np.array(myPoints,ndmin=4)),axis=0)
    
    np.save(AllPoints_file,AllPoints)


end = timer()
print(end-start)


0.05925120000028983


# This code does classification based on spot light search, serial processing, slow. Don't run it
start = timer()
CLconditions=['Neu','Neg','Targ']
radius = 1 #spheres radius


##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

subj_list_one=[subj_list[1]]; 
Nsubj=len(subj_list_one)
all_performance = np.zeros(Nsubj)
for i_sub, subj in enumerate(subj_list_one):
    
    ZstatInfoSubj=ZstatInfo.loc[ZstatInfo['Subj']==subj]
    
    Alldata=LoadAllNii(ZstatInfoSubj,CLconditions) # load Alldata
         
    DataArgDict= {'AllPoints': AllPoints,
              'ZstatInfoSubj':ZstatInfoSubj ,
              'CLconditions': CLconditions, 
              'Alldata': Alldata,
              'pipeSpotLight':pipeSpotLight} 
    SpotLight_performance=np.zeros(AllPoints.shape[0])
    for iFeature in range(AllPoints.shape[0]): #iFeature=40836, range(56500,56610):

        SpotLight_performance[iFeature]= LoopiFeature(iFeature, AllPoints,ZstatInfoSubj ,CLconditions, Alldata, pipeSpotLight)
        
end = timer()
print(end-start)
#SpotLight_performance[56500:56610]

In [20]:
#myPoints = DataArgDict['AllPoints'][iFeature] 
# myPoints = SpherePoints(np.array([30,30,15]) , radius,voxel_dims) # sample good point

#[X, Y, Gr]=MVPA_FeatureExtraction(DataArgDict['ZstatInfoSubj'],DataArgDict['CLconditions'],DataArgDict['Alldata'], myPoints,EEG_features) # extract data for classifer
#[X, Y, Gr]=MVPA_FeatureExtraction(ZstatInfoSubj,CLconditions,Alldata, myPoints,EEG_features) # extract data for classifer
# SpotPerformance=MVPA_Classification(X,Y,Gr, pipeSpotLight])
#MyZstat[Neu_P3_Cz_Peak][0][MyInd]

In [24]:
# This code does classification based on spot light search, parallel processing, fast
# CLconditions=['Neu','Neg']
# Hypothesis_name='Emotion'
CLconditions=['Distractor','Targ']
Hypothesis_name='Decision'
MVPA_out_folder='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/MVPA/'+Hypothesis_name+'_SpotlightR'+str(radius)+'_'+'+'.join(EEG_features)
print(MVPA_out_folder)
if not os.path.exists(MVPA_out_folder):
    os.makedirs(MVPA_out_folder)
    
##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

#subj_list_one=[subj_list[1]]; 
#Nsubj=len(subj_list_one)
Nsubj=len(subj_list)
all_performance = np.zeros(Nsubj)
for i_sub, subj in enumerate(subj_list):#subj_list):
    if (subj=='ET_12102013') & (radius==2): 
        continue  
    print(subj)
    start = timer()

    ZstatInfoSubj=ZstatInfo.loc[ZstatInfo['Subj']==subj]
    Zstattmp=ZstatInfoSubj.loc[ZstatInfoSubj['Run']==1]
    Zstattmp.reset_index(inplace = True, drop = True)
    myFiletmp=Zstattmp['Folder'][0]+Zstattmp['Neg'][0][0] # load a sample file to extract file format
    
    myimg_affine=nib.load(myFiletmp).affine

    Alldata=LoadAllNii(ZstatInfoSubj,CLconditions) # load Alldata
         
    DataArgDict= {'AllPoints': AllPoints,
              'ZstatInfoSubj':ZstatInfoSubj ,
              'CLconditions': CLconditions, 
              'Alldata': Alldata,
              'pipeSpotLight':pipeSpotLight,
              'EEG_features':EEG_features #Eneter None or just delete this line for fMRI only MVPA
                 } 
    SpotLight_performance=np.zeros(AllPoints.shape[0])
    
    aa1= range(AllPoints.shape[0])
    aa2=[DataArgDict]*len(aa1)
    aa=[*zip(aa1, aa2)]

    if __name__ == "__main__":

        pool = Pool(multiprocessing.cpu_count()-2)
        resultsN = pool.map(multi_run_wrapperLoopiFeatureDict,aa)

        resultsN=np.array(resultsN).reshape(voxel_dims)

        nft_img = nib.Nifti1Image(resultsN, myimg_affine)  # save based on the sample file format
        OutFile=MVPA_out_folder+'/MVPA_'+subj+'.nii.gz'
        nib.save(nft_img, OutFile)

    end = timer()
    print(end-start)
# print(resultsN[56500:56510])

C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/MVPA/Decision_SpotlightR2_LPP_Pz_Peak
AB_04152014
644.4465517999924
AM_05082014
740.0299064999999
CJ_06052014
839.7019933999982
CS_04162014
1384.2090813999966
DH_05142014
1350.8730243000027
DS_12132013
2448.802824000013
JK_04252014
1390.4766872999899
KK_04152014
1310.9714410000015
MB_04252014
1433.505448800017
MG_04152014
1119.4455945999944
ML_05122014
1484.518417299987
MM_05132014
1467.3256641000044
NB_05012014
1346.370056899992
NJM_05142014
1551.8466584999987
PD_04162014
1434.7797904000035
RS_04162014
1407.8061988999834
RS_12102013
1999.799068799999
SA_05162014
850.977965099999
TD_05012014
885.8796871000086
TH_12172013
2068.6420330999827
TP_05162014
1384.5223931999935


In [28]:
EEG_features=['P3_Cz_Peak']
CLconditions=['Neu','Neg']
Hypothesis_name='Emotion'
# CLconditions=['Distractor','Targ']
# Hypothesis_name='Decision'
MVPA_out_folder='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/MVPA/'+Hypothesis_name+'_SpotlightR'+str(radius)+'_'+'+'.join(EEG_features)
print(MVPA_out_folder)

C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/MVPA/Emotion_SpotlightR2_P3_Cz_Peak


In [29]:
# This code does classification based on spot light search, serial processing (for trouble shooting) & also issue with ET, radius=2
CLconditions=['Neu','Neg']
Hypothesis_name='Emotion'
# CLconditions=['Distractor','Targ']
# Hypothesis_name='Decision'
MVPA_out_folder='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/MVPA/'+Hypothesis_name+'_SpotlightR'+str(radius)+'_'+'+'.join(EEG_features)
print(MVPA_out_folder)
if not os.path.exists(MVPA_out_folder):
    os.makedirs(MVPA_out_folder)
    
##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

# subj_list_one='ET_12102013'; 
# Nsubj=len(subj_list_one)
Nsubj=1#len(subj_list)
all_performance = np.zeros(Nsubj)
for i_sub, subj in enumerate(['ET_12102013']):
    print(subj)
    start = timer()

    ZstatInfoSubj=ZstatInfo.loc[ZstatInfo['Subj']==subj]
    Zstattmp=ZstatInfoSubj.loc[ZstatInfoSubj['Run']==1]
    Zstattmp.reset_index(inplace = True, drop = True)
    myFiletmp=Zstattmp['Folder'][0]+Zstattmp['Neg'][0][0]
    
    myimg_affine=nib.load(myFiletmp).affine

    Alldata=LoadAllNii(ZstatInfoSubj,CLconditions) # load Alldata
         
    DataArgDict= {'AllPoints': AllPoints,
              'ZstatInfoSubj':ZstatInfoSubj ,
              'CLconditions': CLconditions, 
              'Alldata': Alldata,
              'pipeSpotLight':pipeSpotLight,
              'EEG_features':EEG_features #Eneter None or just delete this line for fMRI only MVPA
                 } 
    SpotLight_performance=np.zeros(AllPoints.shape[0])
    
#     aa1= range(AllPoints.shape[0])
#     aa2=[DataArgDict]*len(aa1)
#     aa=[*zip(aa1, aa2)]

    resultsN = np.zeros(AllPoints.shape[0])
    for iFeature in range(AllPoints.shape[0]): #iFeature=40836, range(56500,56610):
        if (iFeature==74712 and radius==2 and subj=='ET_12102013'): # for ET, radius=2, the code stucks at this voxel
            resultsN[iFeature]=resultsN[iFeature-1]
        else:
            resultsN[iFeature]= LoopiFeatureDict(iFeature, DataArgDict)
            
    
    resultsN=np.array(resultsN).reshape(voxel_dims)

    nft_img = nib.Nifti1Image(resultsN, myimg_affine)
    OutFile=MVPA_out_folder+'/MVPA_'+subj+'.nii.gz'
    nib.save(nft_img, OutFile)

    end = timer()
    print(end-start)
# print(resultsN[56500:56510])

C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/MVPA/Emotion_SpotlightR2_P3_Cz_Peak
ET_12102013
3366.985002600006


If it is needed to modify a funciton and relaod it:

In [ ]:
#del multi_run_wrapperLoopiFeatureDict
#from MVPA_func import multi_run_wrapperLoopiFeatureDict
multi_run_wrapperLoopiFeatureDict = reload(sys.modules["MVPA_func"]).multi_run_wrapperLoopiFeatureDict  # reload() returns the new module
LoopiFeatureDict = reload(sys.modules["MVPA_func"]).LoopiFeatureDict  # reload() returns the new module
